In [1]:
# import libraries
import os
import pandas as pd
import json
import gzip
import fasttext 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd
import re
import numpy as np
import sklearn

In [49]:
test = pd.read_csv('../../src/data/schemafiltereddata/TrainTestTables/CSV/large_tables_seq.csv')

In [50]:
test

,Unnamed: 0,text,name,class
0,0,Xiaomi Wireless Charger Pad Qi 7.5W For iPhone...,name,Product
1,1,"{'availability': 'https://schema.org/InStock',...",offers,Product
2,2,"mytechstyle.com,mytechstyle.com,mytechstyle.co...",brand,Product
3,3,"850,851,2277,850,1892,1892,1893,1893,1566,1172...",mpn,Product
4,4,For Him Outdoors For...,category,Product
...,...,...,...,...
251299,251299,2020- Age Group Training- 13s 2020- Positional...,name,Event
251300,251300,2020-08-21T21:00:00-04:00 2020-08-17T16:45:00-...,enddate,Event
251301,251301,"{'name': 'KIVA Sports Center', 'telephone': '5...",location,Event
251302,251302,Organization Organization Organization Organiz...,performer,Event


In [51]:
test.drop(test[(test['name']=='inalbum') &(test['class']=='MusicRecording')].index, inplace=True)
test.drop(test[(test['name']=='performer') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='eventattendancemode') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='eventstatus') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='owns') &(test['class']=='LocalBusiness')].index, inplace=True)
test["label_complete"] = test["name"] + test["class"]
num_columns = len(set.union(set(test.label_complete.unique())))
test

,Unnamed: 0,text,name,class,label_complete
0,0,Xiaomi Wireless Charger Pad Qi 7.5W For iPhone...,name,Product,nameProduct
1,1,"{'availability': 'https://schema.org/InStock',...",offers,Product,offersProduct
2,2,"mytechstyle.com,mytechstyle.com,mytechstyle.co...",brand,Product,brandProduct
3,3,"850,851,2277,850,1892,1892,1893,1893,1566,1172...",mpn,Product,mpnProduct
4,4,For Him Outdoors For...,category,Product,categoryProduct
...,...,...,...,...,...
251298,251298,http://schema.org/NewCondition http://schema.o...,itemcondition,Product,itemconditionProduct
251299,251299,2020- Age Group Training- 13s 2020- Positional...,name,Event,nameEvent
251300,251300,2020-08-21T21:00:00-04:00 2020-08-17T16:45:00-...,enddate,Event,enddateEvent
251301,251301,"{'name': 'KIVA Sports Center', 'telephone': '5...",location,Event,locationEvent


In [52]:
test.drop(['Unnamed: 0','class'], axis=1, inplace=True)
test['text'] = test.text.apply(lambda x: re.sub("[^0-9a-zA-Z-@]+", " ", x))

In [53]:
labels_new = {'worstratingProduct': 0,
 'acceptsreservationsRestaurant': 1,
 'additionalnamePerson': 2,
 'additionalpropertyProduct': 3,
 'addressHotel': 4,
 'addressLocalBusiness': 5,
 'addressPerson': 6,
 'addressProduct': 7,
 'addressRestaurant': 8,
 'addresslocalityLocalBusiness': 9,
 'addressregionLocalBusiness': 10,
 'affiliationPerson': 11,
 'aggregateratingBook': 12,
 'aggregateratingCreativeWork': 13,
 'aggregateratingHotel': 14,
 'aggregateratingLocalBusiness': 15,
 'aggregateratingProduct': 16,
 'aggregateratingRecipe': 17,
 'aggregateratingRestaurant': 18,
 'alternatenameProduct': 19,
 'alternativeheadlineCreativeWork': 20,
 'areaservedLocalBusiness': 21,
 'articlebodyCreativeWork': 22,
 'audienceProduct': 23,
 'authorBook': 24,
 'authorCreativeWork': 25,
 'authorProduct': 26,
 'authorRecipe': 27,
 'availabilityProduct': 28,
 'bestratingProduct': 29,
 'birthdatePerson': 30,
 'birthplacePerson': 31,
 'bookeditionBook': 32,
 'bookformatBook': 33,
 'brandProduct': 34,
 'breadcrumbProduct': 35,
 'byartistMusicAlbum': 36,
 'byartistMusicRecording': 37,
 'categoryProduct': 38,
 'citystatezipLocalBusiness': 39,
 'colorProduct': 40,
 'commentcountCreativeWork': 41,
 'conditionProduct': 42,
 'contactpointLocalBusiness': 43,
 'contactpointPerson': 44,
 'cookingmethodRecipe': 45,
 'cooktimeRecipe': 46,
 'copyrightholderCreativeWork': 47,
 'copyrightyearCreativeWork': 48,
 'creatorCreativeWork': 49,
 'datecreatedCreativeWork': 50,
 'datecreatedMusicRecording': 51,
 'datemodifiedCreativeWork': 52,
 'datemodifiedRecipe': 53,
 'datepublishedBook': 54,
 'datepublishedCreativeWork': 55,
 'datepublishedMusicRecording': 56,
 'datepublishedProduct': 57,
 'datepublishedRecipe': 58,
 'deathdatePerson': 59,
 'depthProduct': 60,
 'disambiguatingdescriptionProduct': 61,
 'doortimeEvent': 62,
 'durationEvent': 63,
 'durationMusicRecording': 64,
 'emailHotel': 65,
 'emailLocalBusiness': 66,
 'emailPerson': 67,
 'enddateEvent': 68,
 'episodenumberTVEpisode': 69,
 'worksforPerson': 70,
 'familynamePerson': 71,
 'faxnumberLocalBusiness': 72,
 'faxnumberPerson': 73,
 'genderPerson': 74,
 'genreBook': 75,
 'genreCreativeWork': 76,
 'genreMusicRecording': 77,
 'geoHotel': 78,
 'geoLocalBusiness': 79,
 'geoPlace': 80,
 'geoProduct': 81,
 'geoRestaurant': 82,
 'givennamePerson': 83,
 'gtin12Product': 84,
 'gtin13Product': 85,
 'gtin14Product': 86,
 'gtin8Product': 87,
 'gtinProduct': 88,
 'hasmapLocalBusiness': 89,
 'hasmenuRestaurant': 90,
 'headlineCreativeWork': 91,
 'headlineRecipe': 92,
 'heightPerson': 93,
 'heightProduct': 94,
 'homelocationPerson': 95,
 'identifierProduct': 96,
 'worklocationPerson': 97,
 'ingredientsRecipe': 98,
 'inlanguageBook': 99,
 'inlanguageCreativeWork': 100,
 'interactioncountCreativeWork': 101,
 'interactionstatisticCreativeWork': 102,
 'interactiontypeCreativeWork': 103,
 'isbnBook': 104,
 'ispartofRecipe': 105,
 'isrelatedtoProduct': 106,
 'issimilartoProduct': 107,
 'itemconditionProduct': 108,
 'itemlistelementProduct': 109,
 'jobtitlePerson': 110,
 'keywordsCreativeWork': 111,
 'keywordsRecipe': 112,
 'knowslanguagePerson': 113,
 'legalnameLocalBusiness': 114,
 'locationEvent': 115,
 'locationLocalBusiness': 116,
 'mainentityofpageCreativeWork': 117,
 'mainentityofpagePerson': 118,
 'mainentityofpageProduct': 119,
 'mainentityofpageRecipe': 120,
 'makesofferPerson': 121,
 'manufacturerProduct': 122,
 'materialProduct': 123,
 'memberofPerson': 124,
 'menuRestaurant': 125,
 'modelProduct': 126,
 'mpnProduct': 127,
 'nameBook': 128,
 'nameCreativeWork': 129,
 'nameEvent': 130,
 'nameHotel': 131,
 'nameLocalBusiness': 132,
 'nameMusicAlbum': 133,
 'nameMusicRecording': 134,
 'namePlace': 135,
 'nameProduct': 136,
 'nameRecipe': 137,
 'nameRestaurant': 138,
 'nameTVEpisode': 139,
 'nationalityPerson': 140,
 'numberofpagesBook': 141,
 'numtracksMusicAlbum': 142,
 'nutritionRecipe': 143,
 'offerdetailsProduct': 144,
 'offersBook': 145,
 'offersCreativeWork': 146,
 'offersProduct': 147,
 'openinghoursLocalBusiness': 148,
 'openinghoursRestaurant': 149,
 'openinghoursspecificationLocalBusiness': 150,
 'openinghoursspecificationPlace': 151,
 'openinghoursspecificationRestaurant': 152,
 'organizerEvent': 153,
 'partofseriesTVEpisode': 154,
 'paymentacceptedLocalBusiness': 155,
 'widthProduct': 156,
 'performersEvent': 157,
 'performtimeRecipe': 158,
 'postalcodeLocalBusiness': 159,
 'preptimeRecipe': 160,
 'priceProduct': 161,
 'pricecurrencyProduct': 162,
 'pricerangeHotel': 163,
 'pricerangeLocalBusiness': 164,
 'pricerangeRestaurant': 165,
 'publisherBook': 166,
 'publisherCreativeWork': 167,
 'publisherRecipe': 168,
 'ratingvalueProduct': 169,
 'recipecategoryRecipe': 170,
 'recipecuisineRecipe': 171,
 'recipeingredientRecipe': 172,
 'recipeinstructionsRecipe': 173,
 'recipeyieldRecipe': 174,
 'releasedateProduct': 175,
 'reviewLocalBusiness': 176,
 'reviewProduct': 177,
 'reviewRecipe': 178,
 'reviewcountProduct': 179,
 'reviewsProduct': 180,
 'sameasLocalBusiness': 181,
 'sameasPerson': 182,
 'sameasPlace': 183,
 'servescuisineRestaurant': 184,
 'shop-currencyProduct': 185,
 'starratingHotel': 186,
 'streetaddressLocalBusiness': 187,
 'suitablefordietRecipe': 188,
 'telephoneHotel': 189,
 'telephoneLocalBusiness': 190,
 'telephonePerson': 191,
 'telephonePlace': 192,
 'telephoneRestaurant': 193,
 'titleProduct': 194,
 'totaltimeRecipe': 195,
 'trackMusicAlbum': 196,
 'typicalagerangeEvent': 197,
 'versionCreativeWork': 198,
 'weightPerson': 199,
 'weightProduct': 200,
 'founderLocalBusiness': 201
 #'eventattendancemodeEvent': 201,
 #'performerEvent': 202,
 #'inalbumMusicRecording': 203,
 #'eventstatusEvent': 204
 }

In [54]:
test

,text,name,label_complete
0,Xiaomi Wireless Charger Pad Qi 7 5W For iPhone...,name,nameProduct
1,availability https schema org InStock pricecu...,offers,offersProduct
2,mytechstyle com mytechstyle com mytechstyle co...,brand,brandProduct
3,850 851 2277 850 1892 1892 1893 1893 1566 1172...,mpn,mpnProduct
4,For Him Outdoors For Her Kitchen amp Living C...,category,categoryProduct
...,...,...,...
251298,http schema org NewCondition http schema org N...,itemcondition,itemconditionProduct
251299,2020- Age Group Training- 13s 2020- Positional...,name,nameEvent
251300,2020-08-21T21 00 00-04 00 2020-08-17T16 45 00-...,enddate,enddateEvent
251301,name KIVA Sports Center telephone 502-473-120...,location,locationEvent


In [70]:
test['label'] = test.label_complete.map(labels_new)

test.drop(['label_complete', 'name'], axis=1, inplace=True)
test

,text,label
0,Xiaomi Wireless Charger Pad Qi 7 5W For iPhone...,136
1,availability https schema org InStock pricecu...,147
2,mytechstyle com mytechstyle com mytechstyle co...,34
3,850 851 2277 850 1892 1892 1893 1893 1566 1172...,127
4,For Him Outdoors For Her Kitchen amp Living C...,38
...,...,...
251298,http schema org NewCondition http schema org N...,108
251299,2020- Age Group Training- 13s 2020- Positional...,130
251300,2020-08-21T21 00 00-04 00 2020-08-17T16 45 00-...,68
251301,name KIVA Sports Center telephone 502-473-120...,115


In [56]:
test['label'].unique()

array([136, 147,  34, 127,  38,  16, 177,  87, 175,   3, 137, 173,  27,
       174, 172, 160,  46,  58, 195, 170, 171, 168,  92,  91,  25,  55,
        52, 129, 167, 117, 108, 126,  50,  20, 122,  40,  85, 134,  64,
        37, 132,   5, 190,  79, 164, 106,  41,  17, 112, 143, 120, 105,
       188, 128,  24, 104, 166,  54,  75, 131,   4, 186, 163, 130,  68,
       115, 153, 176,  98,  84,  88,  96, 189,  78,  30,  83,  71,  59,
        74, 161, 185, 110, 191,  66, 158, 135,  80, 192,  35,  76, 181,
        93, 140,   6,  14, 199,  95, 113, 121, 162,  28, 200,  15, 148,
        43, 103,  65, 156,  94,  60, 150,  21, 116, 101, 179, 169,  29,
         0,  11,  86,  67,  63,  62,  19, 133,  36, 196, 142,   9,  10,
       187, 159,  39, 138,   8, 193,  82, 152, 198, 151, 100,  47,  48,
        70, 124, 183,   2, 194, 182, 141,  73,  44,  23,  53, 184, 165,
        18, 157, 102, 139, 154,  72,  45, 145,  33, 119,   7,  81, 178,
        77,  51, 107, 123,  97,  89, 144,  99,  42,  57,  12, 15

In [57]:
test[test['label'].isna()]

,text,name,label_complete,label


In [58]:
len (test['label'].unique())

202

In [71]:
test.to_csv('../../src/data/schemafiltereddata/TrainTestTables/CSV/large_tables_prepped.csv', index=False)

In [61]:
#large = pd.read_csv('../../src/data/schemafiltereddata/TrainTestTables/CSV/large_tables_prepped.csv')
testset= pd.read_csv('../../src/data/schemafiltereddata/TrainTestTables/archiv/test_tables_9000_prepped.csv')
testset

,Unnamed: 0,text,label
0,0,Comedy Improv at Arts Garage 2020-12-10 Shows ...,130
1,1,2020-12-10T21 30 00-05 00 2020-12-10T21 30 00-...,68
2,2,name Arts Garage name Bob Carter 8217 s Actor...,115
3,3,name Mod 27 Improv telephone 561-223-1651 ema...,153
4,4,My Front Door Lord Have Mercy Sold Out Wedding...,134
...,...,...,...
49599,49599,url https eutrade-shop com epages f7e4b224-a2...,147
49600,49600,4 5 5 3 5 5 5 4 3 5 3 5 5 5 3 5 1 5 5 4 3 3 5 ...,38
49601,49601,http schema org NewCondition http schema org N...,108
49602,49602,4313042590504 0 4313042595028 0 4313042595066 ...,85


In [64]:
testset = testset.drop(columns={'Unnamed: 0'})
uniques = testset['label'].unique()

In [66]:
larges = (large['label'].unique())
uniques

array([130,  68, 115, 153, 134,  64,  37, 137, 173,  27, 174, 160,  46,
       170,  17, 112,  98, 172,  58, 195, 171, 143, 120, 105,  91,  25,
       103, 132,   5,   9,  10, 187, 159,  39, 136, 147,  38, 108, 106,
        34, 127, 200,  16, 177, 190,  79, 164,  66, 148,  21,  72, 175,
        40, 122, 126, 110,  67,  74, 135,  80, 192,  84,  55,  52, 129,
       167, 117,  15, 176, 131,   4, 189,  78, 138,   8, 193, 184,  82,
       165,  86,  41,  96,  85, 150, 191, 139,  69,   6, 102, 161, 162,
        50,  20,  87, 128,  24, 166,  54,  99,  75, 158, 188,  30,  83,
        71,  59,   2,  88,  70, 182, 119,  94,  43, 114, 156,  60, 154,
       100, 185, 197, 157, 181, 104, 141,  33,  93, 140, 199,  95, 113,
       121,  19,  11, 194,   3,  42,   7,  81, 180,  31,  22,  73, 118,
        90, 149, 123,  14, 107, 198, 163, 179, 169,  29,   0, 178, 133,
        36,  53,  92,  97, 168, 201,  49,  44, 124,  56, 145,  45,  26,
        57,  89, 196,  23, 186,  61, 116,  77,  51,  35,   1, 15

In [69]:
set(uniques)-set(larges)

set()

In [16]:
small = pd.read_csv('../../src/data/schemafiltereddata/TrainTestTables/archiv/small_train_prepped.csv')

In [17]:
small

,text,label
0,Heavenly Sentiments Spray Peaceful White Lilie...,136
1,pricecurrency USD lowprice 202 95 highprice 2...,147
2,reviewcount 4 ratingvalue 5 00 reviewcount 25...,16
3,author from - description Arrangement was mad...,177
4,NewCondition NewCondition NewCondition NewCond...,108
...,...,...
13231,name - address address name name - address na...,166
13232,pricecurrency RUB availability http schema or...,145
13233,2011 0 2016 0 2006 0 2018 0 2016 0 2020 0 2013...,54
13234,736 140 560 208 208 32 288 576 272 264 286 229...,141


In [20]:
sm_list = small['label'].unique()

In [19]:
len(small['label'].unique())

201

In [23]:
set(uniques) - set(sm_list)

{118}